In [1]:
import qiskit
from qiskit import QuantumCircuit, execute
us = qiskit.Aer.get_backend('unitary_simulator')

from numpy import eye, diag, outer, exp, pi, arange, sqrt, array, kron
from scipy.stats import uniform

from numpy.linalg import norm

from tools import revmat

Exercise 1 : Write a circuit for the SWAP gate

In [2]:
SWAP_MATRIX = eye(4)[[0, 2, 1, 3]]

def swap_circ():
    q = QuantumCircuit(2)
    q.cx(0, 1)
    q.cx(1, 0)
    q.cx(0, 1)
    return q

swap_gate = lambda : swap_circ().to_gate()

In [3]:
q = swap_circ()
job = execute(q, backend=us)
V = revmat(job.result().get_unitary())
norm(V - SWAP_MATRIX)

0.0

Exercise 2 : Write a circuit for a controlled phase shift

In [4]:
C_PHASE_MATRIX = lambda t : diag([1]*3 + [exp(1j*t)])

def c_phase_circ(t):
    q = QuantumCircuit(2)
    q.cx(0, 1)
    q.p(-t/2, 1)
    q.cx(0, 1)
    q.p(t/2, 0)
    q.p(t/2, 1)
    return q

c_phase_gate = lambda t : c_phase_circ(t).to_gate()

In [5]:
t = uniform(0, 2*pi).rvs()
q = c_phase_circ(t)
job = execute(q, backend=us)
V = revmat(job.result().get_unitary())
norm(V - C_PHASE_MATRIX(t))

0.0

Exercise 3 : Write a circuit for the Fourier transform

In [6]:
FOURIER_MATRIX = lambda N : exp(2j*pi*outer(arange(N), arange(N))/N)/sqrt(N)

def fourier_circ(n):
    q = QuantumCircuit(n)
    for j in range(n):
        q.h(j)
        for l in range(j+1, n):
            q.append(c_phase_gate(pi*2**(j-l)), [l, j])

    for j in range(n//2):
        q.append(swap_gate(), [j, n-1-j])
    
    return q

fourier_gate = lambda n : fourier_circ(n).to_gate()

In [7]:
n = 7
N = 2**n
q = fourier_circ(n)
job = execute(q, backend=us)
V = revmat(job.result().get_unitary())
norm(V - FOURIER_MATRIX(N))

1.8263113216443953e-13